In [ ]:
import cherrypy
import json
# import adafruit_dht
# from board import D4
import datetime
import time
from base64 import b64encode
from base64 import b64decode
from MyMQTT import MyMQTT
from DoSomething import DoSomething
import os

class Registry(object):
    exposed = True

            
    def __init__(self):
#         self.dht_device = adafruit_dht.DHT11(D4)
        self.test = DoSomething("publisher 1")
        self.test.run()


    def GET(self, *path, **query):  
        if len(path) !=1:
            raise cherrypy.HTTPError(400, 'Wrong path')
         
        #     2. List the names all the models stored in the models folder.
        if path[0] == "list":
            if len(query) != 0:
                raise cherrypy.HTTPError(400, 'Wrong query')
        #create list of models names in "models" directory
            models_list = os.listdir("model/")
        #remove ".tflite" 
            models_list = list(map(lambda x: x.split(".")[0], models_list))
            
        #return list of all models
            output = {
                    'models': models_list
                }

            output_json = json.dumps(output)
            return output_json
        
        # 3. Measure temperature and humidity values with the DHT-11 sensor every 1s. 
# As soon as 6 samples are measured, make a prediction with the model indicated in the request every 1s. 
# Compare the measured vs. the predicted value.
# If the prediction error (absolute error) of temperature (humidity) is greater than a temperature (humidity) threshold, 
# send an alert to a single or multiple remote clients using the SenML+JSON format. 

        elif path[0] == "predict":
            
            if len(query) != 3:
                raise cherrypy.HTTPError(400, 'Wrong query')
                
            #Retriving parameters
            body = cherrypy.request.body.read()
            body = json.loads(body)

            model = body.get('model')
            ttres = body.get('ttres')
            htres = body.get('htres')
            
            #Preparing tflite model
            input_path = "model/" + model + ".tflite"
            interpreter = tflite.Interpreter(input_path)
            interpreter.allocate_tensors()
            input_details = interpreter.get_input_details()
            output_details = interpreter.get_output_details()

            
            temperature_list = []
            humidity_list = []
            
            #4 predictions for testing ( 10 - 6)
            for i in range(10):

                
                if len(temperature_list) >= 6:
                    # (2, 6) array
                    input_serie = np.array([temperature_list[-6:], humidity_list[-6:]]) 
                    interpreter.set_tensor(input_details[0]['index'], input_serie)
    
                    interpreter.invoke()
    
                    output = interpreter.get_tensor(output_details[0]['index'])
                    
                    #need to test
                    t_pred = output[0][0]
                    h_pred = output[0][1]
                    
                #there might be errors (can they occurr separately on each sensor?)
                #maybe need to convert to string, in that case add float in monitoring_client
                t_new = self.dht_device.temperature
                h_new = self.dht_device.humidity
                posixTime = datetime.now().timestamp()
                
                #send alerts
                if abs(t_pred - t_new) > ttres:
                    
                    #alert in senML format: the first record is predicted value, the second is the actual one
                    alert = {"bn": "rpi",
                             "bt": posixTime,
                  "e" : [
                      {"n":  "temperature",  "u":  "°C",  "t":  posixTime,  "v": t_pred},
                        {"n":  "temperature",  "u":  "°C",  "t":  posixTime,  "v": t_new}

                        ]}
                    alert_json = json.dumps(alert)
                    test.myMqttClient.myPublish("/288876/alert", alert_json)
                    
                if abs(h_pred - h_new) > htres:

                    #alert in senML format: the first record is predicted value, the second is the actual one
                    alert = {"bn": "rpi",
                             "bt": posixTime,
                  "e" : [
                      {"n":  "humidity",  "u":  "%RH",  "t":  posixTime,  "v": h_pred},
                        {"n":  "humidity",  "u":  "%RH",  "t":  posixTime,  "v": h_new}

                        ]}
                    alert_json = json.dumps(alert)
                    test.myMqttClient.myPublish("/288876/alert", alert_json)
                    
                #add new values to the lists
                temperature_list.append(t_new)
                humidity_list.append(h_new)
                
                       
                time.sleep(1)
        
                       
        else:
            raise cherrypy.HTTPError(400, 'Wrong path')
            
            
    def POST(self, *path, **query):
        pass
    
    
#      1. Store a tflite model in a local subfolder called models located at the same path of the service script.    
    def PUT(self, *path, **query):
        
        if len(path) !=1 or path[0] != "add":
            raise cherrypy.HTTPError(400, 'Wrong path')
            
        if len(query) >0:
            raise cherrypy.HTTPError(400, 'Wrong query')
        
        #read args
        body = cherrypy.request.body.read()
        body = json.loads(body)
        
        name = body.get('name')
        if name is None:
            raise cherrypy.HTTPError(400, 'name missing')
        
        model = body.get('model')
        if model is None:
            raise cherrypy.HTTPError(400, 'model missing')
            
        #model was b64 encoded, then converted to string: we need to decode it
        tflite_model = b64decode(model)
        
        #save model
        filename = "model/" + str(name) + ".tflite" 
        with open(filename, 'wb') as f:
            f.write(tflite_model)
            
            
            
    def DELETE(self, *path, **query):
        pass
    
    
if __name__== '__main__':
    conf = {'/':{'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    cherrypy.tree.mount(Registry(), '', conf)
    cherrypy.config.update({'server.socket_host':'0.0.0.0'})
    cherrypy.config.update({'server.socket_port':8080})
    cherrypy.engine.start()
    cherrypy.engine.block()
